In [47]:
#Importing libraries
#importing the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
nltk.download('punkt')
import unicodedata 
#from contractions import CONTRACTION_MAP # from contractions.py
import contractions
import re
import nltk
import string 
import spacy
from nltk.tokenize import ToktokTokenizer
from nltk.tokenize import word_tokenize
from pickle import dump
from collections import Counter
from nltk import ngrams
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
#nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package punkt to /home/spavithree/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [48]:
#Reading the columns
# data = pd.read_csv('DataSet/CSV/radiologyReports.csv')
data = pd.read_csv("Path to csv generated in ExtractTextReports.ipynb")
data.shape
data

,study_id,RadiologyReport,subject_id,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,50414267,"There is no focal consolidation, pleural effus...",10000032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,53189527,"The cardiac, mediastinal and hilar contours ar...",10000032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2,53911762,Single frontal view of the chest provided. Th...,10000032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
3,56699142,"The lungs are clear of focal consolidation, pl...",10000032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
4,57375967,PA and lateral views of the chest provided. ...,10000764,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227822,58708861,ET tube ends 4.7 cm above the carina. NG tube...,19999442,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0
227823,57132437,"The lungs are clear, and the cardiomediastinal...",19999733,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
227824,55368167,There has been interval extubation and improve...,19999987,1.0,-1.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN
227825,58621812,Portable supine AP view of the chest provided ...,19999987,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227827 entries, 0 to 227826
Data columns (total 17 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   study_id                    227827 non-null  int64  
 1   RadiologyReport             216305 non-null  object 
 2   subject_id                  227827 non-null  int64  
 3   Atelectasis                 57666 non-null   float64
 4   Cardiomegaly                66799 non-null   float64
 5   Consolidation               23076 non-null   float64
 6   Edema                       65833 non-null   float64
 7   Enlarged Cardiomediastinum  21837 non-null   float64
 8   Fracture                    5831 non-null    float64
 9   Lung Lesion                 8287 non-null    float64
 10  Lung Opacity                58425 non-null   float64
 11  No Finding                  75455 non-null   float64
 12  Pleural Effusion            87272 non-null   float64
 13  Pleural Other 

# Data Preprocessing

- Removing accented characters
- Removal of special charcaters, stopwords, numbers, punctuation, extra whitespaces
- Using Lemmatization to get the root word.
- (Stemming particularly did not work in radiological reports)

In [1]:
def remove_accented_chars(text):
    new_text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return new_text

def remove_special_characters(text):
    # define the pattern to keep
    pat = r'[^a-zA-z0-9.,!?/:;\"\'\s]' 
    return re.sub(pat, '', text)

def remove_numbers(text):
    # define the pattern to keep
    pattern = r'[^a-zA-z.,!?/:;\"\'\s]' 
    return re.sub(pattern, '', text)

def remove_punctuation(text):
    text = ''.join([c for c in text if c not in string.punctuation])
    return text

# def get_stem(text):
#     stemmer = nltk.porter.PorterStemmer()
#     text = ' '.join([stemmer.stem(word) for word in text.split()])
#     return text

def expand_contractions(text):
    text = contractions.fix(text)
    return text

import en_core_web_sm
nlp = spacy.load('en_core_web_sm')
def get_lem(text):
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text



tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')

stopword_list.remove('not')
stopword_list.remove('no')

def remove_stopwords(text):
    # convert sentence into token of words
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    # check in lowercase 
    t = [token for token in tokens if token.lower() not in stopword_list]
    text = ' '.join(t)    
    return text

def remove_extra_whitespace_tabs(text):
    #pattern = r'^\s+$|\s+$'
    pattern = r'^\s*|\s\s*'
    return re.sub(pattern, ' ', text).strip()

def to_lowercase(text):
    return text.lower()


data['RadiologyReport'] = data['RadiologyReport'].astype(str).apply(remove_accented_chars).apply(remove_special_characters).apply(remove_numbers).apply(remove_punctuation).apply(get_lem).apply(remove_stopwords).apply(remove_extra_whitespace_tabs).apply(to_lowercase).apply(expand_contractions);
#data.to_pickle('./processed_data.pkl');
# data.to_csv("preProcessedReport")


In [111]:
data.to_csv("DataSet/CSV/preProcessed.csv")

# Extracting feature vector

In [22]:
!pip install --upgrade gensim

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 24.1 MB 10.5 MB/s            


In [24]:
df = pd.read_csv("DataSet/CSV/preprocessed.csv")
df.head()

,Unnamed: 0,study_id,RadiologyReport,subject_id,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,0,50414267,no focal consolidation pleural effusion pneumo...,10000032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,1,53189527,cardiac mediastinal hilar contour normal pulmo...,10000032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2,2,53911762,single frontal view chest provide no focal con...,10000032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
3,3,56699142,lung clear focal consolidation pleural effusio...,10000032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
4,4,57375967,pa lateral view chest provide lung adequately ...,10000764,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN


# Create a corpus of all words in the document and get the vector format

In [25]:
#Drop empty rows in radiology report column
df = df.dropna(subset=['RadiologyReport'])

In [26]:
df.shape

(216303, 18)

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 216303 entries, 0 to 227826
Data columns (total 18 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Unnamed: 0                  216303 non-null  int64  
 1   study_id                    216303 non-null  int64  
 2   RadiologyReport             216303 non-null  object 
 3   subject_id                  216303 non-null  int64  
 4   Atelectasis                 53064 non-null   float64
 5   Cardiomegaly                60016 non-null   float64
 6   Consolidation               20818 non-null   float64
 7   Edema                       61993 non-null   float64
 8   Enlarged Cardiomediastinum  17003 non-null   float64
 9   Fracture                    5447 non-null    float64
 10  Lung Lesion                 7631 non-null    float64
 11  Lung Opacity                53933 non-null   float64
 12  No Finding                  74175 non-null   float64
 13  Pleural Effusi

In [28]:
df.drop('Unnamed: 0',axis='columns', inplace=True)

In [29]:
df

,study_id,RadiologyReport,subject_id,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,50414267,no focal consolidation pleural effusion pneumo...,10000032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,53189527,cardiac mediastinal hilar contour normal pulmo...,10000032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2,53911762,single frontal view chest provide no focal con...,10000032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
3,56699142,lung clear focal consolidation pleural effusio...,10000032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
4,57375967,pa lateral view chest provide lung adequately ...,10000764,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227822,58708861,et tube end cm carina ng tube stomach lung oth...,19999442,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0
227823,57132437,lung clear cardiomediastinal silhouette hila n...,19999733,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
227824,55368167,interval extubation improved lung volume compa...,19999987,1.0,-1.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN
227825,58621812,portable supine ap view chest provide demonstr...,19999987,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [119]:
df.to_csv('test.csv')

In [30]:
df.reset_index(inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216303 entries, 0 to 216302
Data columns (total 18 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   index                       216303 non-null  int64  
 1   study_id                    216303 non-null  int64  
 2   RadiologyReport             216303 non-null  object 
 3   subject_id                  216303 non-null  int64  
 4   Atelectasis                 53064 non-null   float64
 5   Cardiomegaly                60016 non-null   float64
 6   Consolidation               20818 non-null   float64
 7   Edema                       61993 non-null   float64
 8   Enlarged Cardiomediastinum  17003 non-null   float64
 9   Fracture                    5447 non-null    float64
 10  Lung Lesion                 7631 non-null    float64
 11  Lung Opacity                53933 non-null   float64
 12  No Finding                  74175 non-null   float64
 13  Pleural Effusi

In [31]:
corpus = []

for row in range(0,len(df)):
    listOfWords = str(df['RadiologyReport'][row]).split(" ")
    corpus.append(listOfWords)

print(len(corpus))

216303


In [32]:
#word2Vec model
from gensim.models import Word2Vec, FastText
w2vModel = Word2Vec(corpus, min_count=1)

In [33]:
print(w2vModel)

Word2Vec(vocab=13656, vector_size=100, alpha=0.025)


In [34]:
print(w2vModel.wv.key_to_index["pneumonia"])

24


In [35]:
# words = list(w2vModel.wv.vocab)
# print(words)

In [36]:
file = 'DataSet/CSV/radiologyReports_embd.txt'
w2vModel.wv.save_word2vec_format(file, binary = False)

Now that we have our word2vec model, how do we use it to transform each documents into a feature vector? In order to convert a document of multiple words into a single vector using our trained word2vec, we take the word2vec of all words in the document, then take its mean.
We'll define a transformer (with sklearn interface) to convert a document into its corresponding vector

https://www.kaggle.com/jhantu/basic-nlp-bag-of-words-tf-idf-word2vec-lstm

In [37]:
class MyTokenizer:
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        transformed_X = []
        for document in X:
            tokenized_doc = []
            for sent in nltk.sent_tokenize(document):
                tokenized_doc += nltk.word_tokenize(sent)
            transformed_X.append(np.array(tokenized_doc))
        return np.array(transformed_X)
    
    def fit_transform(self, X, y=None):
        return self.transform(X)

class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(word2vec.wv.vectors[0])

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = MyTokenizer().fit_transform(X)
        
        return np.array([
            np.mean([self.word2vec.wv[w] for w in words if w in self.word2vec.wv]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])
    
    def fit_transform(self, X, y=None):
        return self.transform(X)

In [38]:
mean_embedding_vectorizer = MeanEmbeddingVectorizer(w2vModel)
mean_embedded = mean_embedding_vectorizer.fit_transform(df['RadiologyReport'])

<ipython-input-37-6dbee262e849>:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(transformed_X)


In [39]:
df['featureVector'] = list(mean_embedded)

In [40]:
df

,index,study_id,RadiologyReport,subject_id,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices,featureVector
0,0,50414267,no focal consolidation pleural effusion pneumo...,10000032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,"[1.0923693, 0.6238863, 0.12972775, -0.21720658..."
1,1,53189527,cardiac mediastinal hilar contour normal pulmo...,10000032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,"[1.4372452, 0.106786795, 0.46272925, -0.036505..."
2,2,53911762,single frontal view chest provide no focal con...,10000032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,"[0.9545043, -0.37497592, -0.07248781, -0.12124..."
3,3,56699142,lung clear focal consolidation pleural effusio...,10000032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,"[1.2446634, 0.10751399, 0.16127141, 0.22847979..."
4,4,57375967,pa lateral view chest provide lung adequately ...,10000764,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,"[0.38414526, -0.21171202, 0.31989226, 0.069970..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216298,227822,58708861,et tube end cm carina ng tube stomach lung oth...,19999442,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,"[-0.780467, -0.95510125, 0.45822012, -0.104858..."
216299,227823,57132437,lung clear cardiomediastinal silhouette hila n...,19999733,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,"[1.0258565, 0.36036262, 0.8081508, 0.093050085..."
216300,227824,55368167,interval extubation improved lung volume compa...,19999987,1.0,-1.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,"[-0.13144848, -0.27976665, 0.2008919, -0.33302..."
216301,227825,58621812,portable supine ap view chest provide demonstr...,19999987,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,"[-0.65706515, -0.973761, -0.16906758, -0.03161..."


In [41]:
df['featureVector']
#df.to_csv('file_name.csv')


0         [1.0923693, 0.6238863, 0.12972775, -0.21720658...
1         [1.4372452, 0.106786795, 0.46272925, -0.036505...
2         [0.9545043, -0.37497592, -0.07248781, -0.12124...
3         [1.2446634, 0.10751399, 0.16127141, 0.22847979...
4         [0.38414526, -0.21171202, 0.31989226, 0.069970...
                                ...                        
216298    [-0.780467, -0.95510125, 0.45822012, -0.104858...
216299    [1.0258565, 0.36036262, 0.8081508, 0.093050085...
216300    [-0.13144848, -0.27976665, 0.2008919, -0.33302...
216301    [-0.65706515, -0.973761, -0.16906758, -0.03161...
216302    [-0.9398432, -0.789139, 0.14690985, -0.4075524...
Name: featureVector, Length: 216303, dtype: object

In [42]:
#removal of nan and -1
df=df.replace(np.nan, 0)
df=df.replace(-1,0)
dfpractice=df

In [43]:
dfpractice[(dfpractice['Atelectasis'] ==0)&(dfpractice['Cardiomegaly'] ==0)&(dfpractice['Consolidation'] ==0)&(dfpractice['Edema'] ==0)&(dfpractice['Enlarged Cardiomediastinum'] ==0)&(dfpractice['Fracture'] ==0)&(dfpractice['Lung Lesion'] ==0)&(dfpractice['Lung Opacity'] ==0)&(dfpractice['No Finding'] ==0)&(dfpractice['Pleural Effusion'] ==0)&(dfpractice['Pleural Other'] ==0)&(dfpractice['Pneumonia'] ==0)&(dfpractice['Pneumothorax']==0)&(dfpractice['Support Devices'] ==0)]


,index,study_id,RadiologyReport,subject_id,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices,featureVector
11,12,58219844,lung volume low result crowding bronchovascula...,10000935,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.3352366, -0.06574851, 0.2294427, -0.2162716..."
12,13,50985099,compare chest radiograph since recently one pr...,10000980,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.055444688, 0.15137808, 0.15233454, 0.299596..."
22,23,53447138,lung volume normal no evidence tb parenchymal ...,10001122,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.28008494, -0.019731807, 0.16432877, 0.35901..."
44,45,53880659,hyperinflation no pneumothorax effusion consol...,10001884,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.20454969, -0.336209, -0.6185337, 0.5214739,..."
69,72,55941092,pa lateral view chest provide midline sternoto...,10002013,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.87021583, -0.015933696, -0.3299211, 0.22603..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216061,227566,50560539,compare previous radiograph no relevant change...,19992507,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[-0.080859534, -0.55683917, 0.8996518, 0.39365..."
216069,227574,50634814,ap lateral view chest perform patient position...,19992875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.14224017, -0.33869553, -0.19892201, -0.1873..."
216121,227633,51369728,portable ap upright chest radiograph obtain lu...,19994730,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.4544339, 0.31389043, -0.008468376, -0.06412..."
216256,227774,59569283,cardiac mediastinal hilar contour appear stabl...,19997911,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[-0.56958055, -0.40608168, 0.1932123, -0.34176..."


In [44]:
df_new=dfpractice.drop(dfpractice[(dfpractice['Atelectasis'] ==0)&(dfpractice['Cardiomegaly'] ==0)&(dfpractice['Consolidation'] ==0)&(dfpractice['Edema'] ==0)&(dfpractice['Enlarged Cardiomediastinum'] ==0)&(dfpractice['Fracture'] ==0)&(dfpractice['Lung Lesion'] ==0)&(dfpractice['Lung Opacity'] ==0)&(dfpractice['No Finding'] ==0)&(dfpractice['Pleural Effusion'] ==0)&(dfpractice['Pleural Other'] ==0)&(dfpractice['Pneumonia'] ==0)&(dfpractice['Pneumothorax']==0)&(dfpractice['Support Devices'] ==0)].index)    

In [45]:
df_new

,index,study_id,RadiologyReport,subject_id,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices,featureVector
0,0,50414267,no focal consolidation pleural effusion pneumo...,10000032,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,"[1.0923693, 0.6238863, 0.12972775, -0.21720658..."
1,1,53189527,cardiac mediastinal hilar contour normal pulmo...,10000032,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,"[1.4372452, 0.106786795, 0.46272925, -0.036505..."
2,2,53911762,single frontal view chest provide no focal con...,10000032,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,"[0.9545043, -0.37497592, -0.07248781, -0.12124..."
3,3,56699142,lung clear focal consolidation pleural effusio...,10000032,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,"[1.2446634, 0.10751399, 0.16127141, 0.22847979..."
4,4,57375967,pa lateral view chest provide lung adequately ...,10000764,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.38414526, -0.21171202, 0.31989226, 0.069970..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216298,227822,58708861,et tube end cm carina ng tube stomach lung oth...,19999442,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,"[-0.780467, -0.95510125, 0.45822012, -0.104858..."
216299,227823,57132437,lung clear cardiomediastinal silhouette hila n...,19999733,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,"[1.0258565, 0.36036262, 0.8081508, 0.093050085..."
216300,227824,55368167,interval extubation improved lung volume compa...,19999987,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[-0.13144848, -0.27976665, 0.2008919, -0.33302..."
216301,227825,58621812,portable supine ap view chest provide demonstr...,19999987,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,"[-0.65706515, -0.973761, -0.16906758, -0.03161..."


In [46]:
df.to_csv('DataSet/CSV/FinalRadiologyReportsLabeled.csv')